In [1]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.4 MB/s eta 0:00:0

In [2]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [3]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
language = "english"

In [5]:
model_name = "sshleifer/distilbart-xsum-12-3"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set model parameters or use the default
# print(model.config)

# tokenization
encoder_max_length = 1024 #256  # demo
decoder_max_length = 100 #64

In [6]:
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
from datasets import load_dataset
data = load_dataset("knkarthick/AMI")

# Take a look at the data
for k, v in data["train"][0].items():
    print(k)
    print(v)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--AMI-e0c25640a9a75f54/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

id
30
dialogue
Speaker A: Cool. Do you wanna give me the little cable thing? Yeah. Cool. Ah, that's why it won't meet. Okay, cool. Yep, cool. Okay, functional requirements. Alright, yeah. It's working. Cool, okay. So what I have, wh where I've got my information from is a survey where the usability lab um observed remote control use with um a hundred subjects and then they gave them a questionnaire. Um so it was all about, you know, how people feel about the look and feel of the remote control, you know. What's the most annoying things about remote controls and um the possibility of speech recognition and L_C_D_ screens in remote control. Not that they actually gave me any answers on the L_C_D_ screens, so I should have taken that bit out, but anyway. Um okay, so. What they found is that people don't like how current remote controls are, so you know, definitely you should be looking at something quite different. Um seventy five percent of users find most remote controls ugly. Uh the ot

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 209
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 28
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 42
    })
})

In [13]:
data = datasets.concatenate_datasets([data["train"], data["validation"], data["test"]])

In [14]:
data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 279
})

In [ ]:
def flatten(example):
    return {
        "dialogue": example["dialogue"],
        "summary": example["summary"],
    }

data.map(flatten, remove_columns=["id"])[0]

In [21]:
def flatten(example):
    return {
        "dialogue": example["dialogue"],
        "summary": example["summary"],
    }


def list2samples(example):
    dialogues = []
    summaries = []
    for sample in zip(example["dialogue"], example["summary"]):
        if len(sample[0]) > 0:
            dialogues += sample[0]
            summaries += sample[1]
    return {"dialogue": dialogues, "summary": summaries}


dataset = data.map(flatten, remove_columns=["id"])
#dataset = dataset.map(list2samples, batched=True)

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

In [22]:
train_data_txt[0]

{'dialogue': "Speaker A: Okay. Mm. Very fun. Oh right. Okay. Mm. Okay. Sure. We made three for you. Um one's based on the banana, one's based on the tomato and the other one is st Look. Oh yeah, well yeah, we had v some red left over. So. Okay, so this is the um non to non uh no buttons one, or as mm few buttons as possible, mainly speak recognition. The yellow there is the um the slogan, yeah, that we need to incorporate, it's very simple. If you do need buttons, you can flip it over, and there's some there, um but mainly it's speech recognition. Yeah. Yeah and yeah they might project things onto the screen which you can do on there. Mm I'm not sure about that. Um and this one is the one w more like the one w that we looked at earlier. Yeah, you guys can have a look at that if you want. Yeah, sure. Um that one is Oh no, it's delicate. That's that's already got its stand that one. That's it stand. It does also lie flat, but that's the that yellow stand there represents the the charging

In [23]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["dialogue"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [24]:
train_data['input_ids'][0]

[0,
 29235,
 4218,
 83,
 35,
 8487,
 4,
 256,
 119,
 4,
 12178,
 1531,
 4,
 5534,
 235,
 4,
 8487,
 4,
 256,
 119,
 4,
 8487,
 4,
 9136,
 4,
 166,
 156,
 130,
 13,
 47,
 4,
 12698,
 65,
 18,
 716,
 15,
 5,
 23611,
 6,
 65,
 18,
 716,
 15,
 5,
 20406,
 8,
 5,
 97,
 65,
 16,
 1690,
 6893,
 4,
 5534,
 11380,
 6,
 157,
 11380,
 6,
 52,
 56,
 748,
 103,
 1275,
 314,
 81,
 4,
 407,
 4,
 8487,
 6,
 98,
 42,
 16,
 5,
 7252,
 786,
 7,
 786,
 37463,
 117,
 14893,
 65,
 6,
 50,
 25,
 15408,
 367,
 14893,
 25,
 678,
 6,
 4412,
 1994,
 4972,
 4,
 20,
 5718,
 89,
 16,
 5,
 7252,
 5,
 16760,
 6,
 11380,
 6,
 14,
 52,
 240,
 7,
 14518,
 6,
 24,
 18,
 182,
 2007,
 4,
 318,
 47,
 109,
 240,
 14893,
 6,
 47,
 64,
 11113,
 24,
 81,
 6,
 8,
 89,
 18,
 103,
 89,
 6,
 7252,
 53,
 4412,
 24,
 18,
 1901,
 4972,
 4,
 8976,
 4,
 8976,
 8,
 11380,
 51,
 429,
 695,
 383,
 2500,
 5,
 2441,
 61,
 47,
 64,
 109,
 15,
 89,
 4,
 256,
 119,
 38,
 437,
 45,
 686,
 59,
 14,
 4,
 12698,
 8,
 42,
 65,
 16,
 5,
 65,
 885,
 5

In [25]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-25-16757958f45a>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


## Training

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=10,#50,
    save_total_limit=3,
    push_to_hub=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [27]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_AMI",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "AMI",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run" + "_" + current_time

wandb: Currently logged in as: jainr3 (rahul96jain). Use `wandb login --relogin` to force relogin


In [28]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 8.125521659851074,
 'eval_rouge1': 15.4284,
 'eval_rouge2': 2.3512,
 'eval_rougeL': 11.9244,
 'eval_rougeLsum': 13.9912,
 'eval_gen_len': 22.5357,
 'eval_runtime': 113.9426,
 'eval_samples_per_second': 0.246,
 'eval_steps_per_second': 0.061}

In [29]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
50,7.264100


TrainOutput(global_step=63, training_loss=7.008646647135417, metrics={'train_runtime': 1071.6032, 'train_samples_per_second': 0.234, 'train_steps_per_second': 0.059, 'total_flos': 77704318156800.0, 'train_loss': 7.008646647135417, 'epoch': 1.0})

In [30]:
trainer.evaluate()

{'eval_loss': 6.141812801361084,
 'eval_rouge1': 21.6597,
 'eval_rouge2': 4.161,
 'eval_rougeL': 15.535,
 'eval_rougeLsum': 18.0175,
 'eval_gen_len': 28.6071,
 'eval_runtime': 125.6039,
 'eval_samples_per_second': 0.223,
 'eval_steps_per_second': 0.056,
 'epoch': 1.0}

In [31]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,▁█
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁██
train/global_step,▁▇██


In [ ]:
trainer.push_to_hub()

In [32]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["dialogue"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (62) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [34]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
print("\nSource dialogue:\n")
print(tabulate(list(enumerate(test_samples["dialogue"])), headers=["Id", "Dialogue"]))

  Id  Summary after                                                                                                                                                                                                                                                      Summary before
----  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------
   0  This is the agenda for today's kick-off meeting.                                                                                                                                                                                                                   The BBC's Reality Check team has been talking to some

## Inference test

In [35]:
result = " Hi, this is Eric Johnson. It's February 18, 2021, and this is the engineering key review at GitLab. So I've got number four in the agenda, which is a proposal to break up this meeting into four department key reviews. So currently this is engineering. Development quality, security, and UX infrastructure and support to their own key reviews already. I have the reasons why increased visibility able to go deeper. Increased the objectivity with which my reports can manage their groups, allow me more time to focus on new markets and allow me to shift into more of a question asker mode, then generating content and answering questions in these meetings. And to avoid adding three net new meetings to stakeholders counters, I propose we do a sort of two month rotation. So month one development quality go month two security and UX would go. How do people feel about that proposal? I think in the group conversations it's working really well. So I'm supportive and this is the smallest thing. Maybe we need four meetings a month, like it's the biggest department, it's super central. But you proposed this. I don't. I could see either way. So let's stick with the proposal. Go, we'll try it and we'll we can be flexible. I mean, development is larger. Maybe they go more frequently or something. I'll see how it goes. All right. And then I've got number five, which is we've got R&D overall MR rate. And we also have R&D wider MR rate, both as top level KPIs for engineering. So the difference between them in the simplest sense is that R&D wider MR rate includes both community contributions and community MRs. The problems I see with this are that one, the wider MR rate, the one that includes internally and external MRs, it duplicates the overall MR rate, which is. The wider MR rate should just be external, right? And then overall should be narrow plus wider. Oh, yeah. Like we say the wider community. Right, right, right. Okay. So there's. I have to check the taxonomy. Well, like, can you can you confirm? That's that's that's, that's, that's reasoning is my understanding as well. Yeah, I believe water MR rate just captures community contributions. Only and no internal. Yeah, and the reason we measure data is that like one of the most likely failure modes is that we lose the community. Yeah. So we're get, or we're gets goofy is the. When you look at a specific team within the company, there could be contributions outside of that that aren't community contributions. They would be viewed community contributions by that group, but effectively they're not from outside the company. So that's why we use wider to kind of reflect that. And narrow is very specific to the team. But are you saying that if someone in plan contributes to verify its viewed as wider? Not quite that plan plan and verify are just fine. It's when you look at like the development versus infrastructure infrastructure will oftentimes contribute to developments work, but it won't be counted as MRs. Yeah, that's a, that's a potential bit of funkiness that we should talk about separately. I didn't have that in my sort of critique of this, but that that doesn't necessarily make intuitive sense to me. So, so then I think part of my critique of this can be thrown out because it's not as duplicative as I thought, but I still think there's a problem with R&D wider MR rate, which is this thing doesn't really move. In part because it's a rate, so it feels like the way to drive this up is to specifically drive community authors to contribute more than one MR per month. That's how this moves up because it's a productivity rate like we use internally. And that doesn't necessarily feel like the right thing because there's scenarios in which this goes up. We've actually got less contributions overall and less contributors overall. Wait, wait, wait a second. So you're saying that R&D wider MR rate is number MRs per external contributor. Oh my goodness, that should not be the thing. It should be contributions for GitLab team member. So to contribute those, the thing above the division is the external ones. The thing below is the number of team members at GitLab. Is that the case, Lily? I'm checking right now. I think so in our MRs per team member. So unless we start calling people outside the company team member, then it shouldn't be that. Yeah, just clarifying here. So our numerator is community contributions and then our denominator is GitLab team members. So it's not per external member. So we've got what we're doing there, Eric, is we're not trying to say how many MRs does someone send if they send something? How many MRs from external do we get for the size of our organization? Sorry, I have a childhood emergency outside the door. So maybe explain the context behind this. The context is as we grow as a company, we should make sure we keep the community up. Like the logical thing is for the community to flatline and the size of the org to go and before you know, you've kind of outgrown the wider community. Yeah, what I'm seeing is we created this pre-sophisticated taxonomy with prefixes and postfixes to talk about these things. But in reality, we've only got two of them. And it's we keep forgetting, we have a hard time discussing this thing. So I'd rather just name them simply two names for what they are rather than using the taxonomy. But also like in F, I have this proposal of like, what if we just track as a KPI of the percentage of total MRs that come from the community over time? And we would see that drop. I love that. Let's do that instead. But the thing, the thing why we have this complex thing is because you can game that you want to game that you just produce fewer MRs with the engineers that get one. So if you drive that really hard and say this is your number one goal, it's very easy to achieve is just tell all your engineers to produce half. Yeah, so we have we have different metrics to prevent that from happening the same way that like support SLAs and asset kind of buttress one another. I think we're we're robust to that. But simplifying this would make these conversations. If you as our CTO don't even understand them, we went overboard. So I'm supportive. And I forgot and I was agreeing this stuff this morning and like, there's a problem with this and then you just remind me of the context. So yeah, I can't really have my head. A percentage that come from the community. I love that. It's what all our investors ask about. Let's do that. Okay. Cool. So, um, Lily, if you can work with Max to make that transition of the great. And I'll hold the one that we're talking about. That's from the mulberry. Thanks. I'm on the call. Sorry, it was a bit late. Timeline. We do have PIs on the raw number of community MRs and we can make the shift. And why they're confirming from the definition, I think why they only counts for community. And that's that's what the definition is. Cool. All right. So number six, then Christopher. Sorry, I was looking up to see if I had the percentage graph because I think we played around with this at one point and had a draft of that. Probably about five months back, if I can remember, Lily. Just in FYI month of February, if you were looking at any particular metrics, particularly in development at MR rate. We haven't had updates in four days. There's apparently a lag issue that's been problematic for the data team to basically get updated metrics and they're working on that. Okay. I'm sorry, Max. You get the next one. So yeah, there's some some color there on the on the medication, the lag later on. On to on to seven. We continue. I said FYI in addition to the TBI status. I'm sorry. I wanted to just touch on the postgres replication issue there real quick. I've been trying to get my arms wrapped around it. Um, do we have the right attention to this? This is kind of Eric. I don't know if you were commenting on. Hitting towards this in the last meeting around some of the infrastructure improvements on the product side. I'm just not quite sure whose responsibility is to focus on getting a handle on some of the constraints we have on replication. So what I was mentioning in the in the product queue review about an hour ago is I think is sort of like unrelated. And so I think the DRI needs to be your kind of data engineering team. But of course there's a dependency on infrastructure because that's where the data is being piqued from. They they do on that data source. So let's say for the replication lag on that slave host where I'm sorry not the on the the secondary host where the data is being pulled from like infrastructure would be the DRI for that. And so any escalations but well all those. And I know we have an action plan for that as far as creating another dedicated host just for the data team to pull from. I did ask I saw that issue and I did talk to Craig gums a little bit as well on the database side just to see if there's some database improvements and I'm still trying to figure out, you know, if it's truly just dedicated computation old sort of resource a server. Or if there's actually some some database tuning that needs to occur, do you have a sense of that. There's so I'd say it's it's three different things. It's having a dedicated host that doesn't have conflicting query traffic coming from other other workloads. There are some tuning performance or tuning improvements to be made. And then there's also improvements in and this is where it does maybe relate a little bit to what the topic was in the last review basically the overall demand on the database layer from from dot com activities and like improving those. It's definitely not just one of those things that one of the most specific actions we're going to take though is separating out and having a dedicated host so that we're just dealing with the profile of the data engineering traffic on there. And not having conflicting query. And conflicting queries affect the ability to update the replication. Steve, I definitely want to partner with you on this one because I think the the demand on those databases is only going to increase. It's not going down. And I think we need to get. I'm still unclear on where to focus and to get the biggest bang for the buck. I think of the computational resource dedication that's going to be a good thing, but I'll probably going to squeeze the balloon. Next area will will honor itself. Okay, I'll put into the infricky review for next week. Okay, on this issue. Thank you, Steve. So then back to mecon 7 or. Yes. Thank you. And Rob was in these assassins that this morning as well. We have the attention there. Number seven, just provide provide update on previous conversations. We continue to improve defect tracking and against SLOs. There is a first iteration P.I that we are experimenting to show percentage of defects meeting. And we are going to be able to see the SLOs key findings as ones are hovering at 80% as to at 60. We've been focused mostly on S1 as to this point as hence why S3 and S4s are a lower. And this will likely be the case. We are also in point B are working on the measurement for average open box age. This would give us a whole picture of what's what's left. So the age goes up and down. We are cleaning the backlog. The average age should go down as well. There's no P.I yet, but I just want to update and show up beyond this. It's not a off track. Number C Craig on S2. Yeah, just looking through the charts. And I noted that there's a spike in meantime to close. Yeah, and just want to see if you had any insight into that for us. This is the S2. Yeah, this is where the point B on age and supplemental charts in the back end helps. So I haven't seen a dip in age nor the count overall. I think it's the latter. We need to dig in a bit deeper in that. And also the data lag. I would like to re-evaluate when we have a whole picture of when everything is synced in as well. Christie, you have some insights. Yeah, I'm just wondering if part of this could be the fact that we changed the severity across the board for MRs to S2. And so we may have some older bugs in there that hadn't been addressed because they were at a lower severity. Now we've moved them to S2 and maybe that caused a little spike. That could be the case that we did in a limited fashion. It won't be a huge volume. We also iterated after that to pin on priority since product owns prioritization. So I wouldn't account it entirely to that. I mean, this isn't the infertile key review, but. I know that they've gotten backed up on those issues. So if some some good portion of those are infertile created or related, then that may be lifting it as well. I can take the deeper dig in and then provide an update next time. I think we need extra debug slicing of the data here. So you would like to go to point eight. Yeah, we are now measuring S1 as to SLO achievement with closed bugs. But if you then look at the number of bugs, it's exponential growth. And then it will be trivial to us to achieve 100% SLO achievement. If you just look at close bugs, even though there would be a major problem in the company, 99% of all bugs are overdue. As long as I only close ones that are still within the SLO, I'll have great achievement. So I think we shouldn't be looking at the closed bugs. I think we should be looking at open bugs. The entire population or percentage of those is within the SLO time. I think we're doing it the wrong way. Thanks for the thanks for the feedbacks that. Hence why we wanted to have the average age to measure what's outside in the open. And then make this iteration to measure also measure focus on the age of all opened, including open bugs. This is also something we have discussed with Christopher in the next iteration as well. And we happy when happy to adjust. So the key. Yes, go ahead. So average age would get closer to it. It's not what I'm proposing. What I'm proposing is off the open bugs are percentages outside of SLO. So display it as a percentage. You do now just do it about the open bugs month or close. Got it. Okay. The exceeding SLO for open bugs. Yeah, or open bugs that are within SLO. So you have a chart that should go up and to the right like everything else. Sounds great. We can take it to the next data metrics work stream to deliver this. Cool. Thanks. Is it pretty good? Yeah, I like it. You make you have to figure out how to. Because we like to be able to have charts that we can historically reconstruct if we need to. So in tickets close out, you need to go through their history to figure out at this time when it was open. Did it breach the SLO or not? That's a good point. This might be much harder computationally. And so I totally respect if we can do it for that reason. Yeah, I think that's a good point. Yeah, I think that's a good point. Cool. Nine. Yeah, I just wanted to ask the team like I went through all the key meeting metrics. Everything looked in line with prior periods and look good. Is there anything the team wants to call out? It's mostly that we should be watching. Yeah, I'll call out. So the good news is in Q4. We had our smallest decline over several quarters. So we only went down by a tenth of a point. The quarter previous was point six or six tenths of a point and the quarter before that was a full point. So. We see this as an improvement, even though it was still a decline, but it's still a decline. Obviously, we want this actually tracking in an upward direction. We also don't have enough data to know whether or not this is an actual real trend up. So I'm optimistic. I think this is a good thing. We have had a much keener focus on us over the past several quarters. And that's why I think, okay, the work that we've done. I think actually is catching up and getting noticed in sus, but we got to keep an eye on it. We can't. We cannot assume that that's the case. Yeah, and the bug discussion above just kind of points out that like we have an underlying problem right now in our metrics measurement. So if we change the measurements to reflect that, then hopefully we're in good shape. If we don't and we flat line and address it so that we flat line open S1s and S2s, you will see a temporary jump in and above SLOs. As we clear out that back, I'll go over that period of time. So we have a point C, which is similar to infrastructure. We need to get more security work prioritized or hearing that from the team, but neither that problem or that activity is sort of currently reflected in our security metrics. So we have some work to do long term to make sure the we see things like that in the metrics and the measurements that we're making. So back to you said 10. So now that Nero MR rate seems significantly below target. And maybe I helped that it would bounce back from December. I think it bounced back, but not back on target. Any any context there? What's going on? Yeah, so with family and friends days, we actually had some heavier of vacation days. And we historically have one thing to know is that we are actually at a higher MR rate. If you look, if you go back the last 18 months, we're actually at a higher Nero MR rate than we were back in each month this year. So if you compare October to October, November to November and January, I'm sorry, October, November, December and January, comparatively to last year, which will find is we're between a half point and a 1.5. MR rate above where we were in the month of previous year. That's great contact. Thank you, Chris, for good work. Yeah, so the expectation is is that February is a short month. We were at I think 16 work days with friends and family day and other things. Obviously seven carriages in Texas doesn't help things either for the folks who are working in Texas. But hopefully the rest of the team is being effective. I was hoping to see a better result right now, but with four or five days, particularly around release week. That's usually when we see do do see a little bit higher activity. So that's not accounted for yet. But you know, Marches, Marches when I'm expecting kind of see a real rebound much like we did last year. Awesome. Thanks. The other context I'll give is we now do time series targets. So when we change the target, you'll see they're reflected in the line. So if we were to look back historically here, the goal here was actually lower and Christopher was ambitious. So we kept raising it. We kept meeting that. So it should stare step here and we could go back and reconstruct that if we really wanted to. And then ignoring the sort of the seasonal dip here, we raised it to I think 11. And then we realize we're kind of hitting that point of doing and raising returns and the right thing to do business wise. And this is in our FY 22 direction is hold the line at productivity. Let's start to raise other things related to quality security, availability and what not. So that's kind of what you are, you're seeing this bump is we raised it and we realized, okay, that's not the we shouldn't raise it anymore. And we brought it back down to 10. So 10 will be the dark going forward. I'm going to try to get better a lot of other things while preventing this from dipping. And just want to call out that it's not necessary. Like, and higher MR rates should also help to address security and quality and other things because you're more productive. So you can fix more things so it's not necessarily opposite. But I agree with that. Let's hold the line 10 is then is a great number and and focus on other indicators to improve that makes it on a sense. Cool. Well said. All right, that's the agenda. Anyone want to vocalize anything else? Great. Well, thanks everybody. And I'm going to go check on my four year old and see if she got what whatever she needed. So cheers and toxic."

In [37]:
def generate_inference_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


#model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#test_samples = validation_data_txt.select(range(16))

#summaries_before_tuning = generate_inference_summary(test_samples, model_before_tuning)[1]
#summaries_after_tuning = generate_inference_summary(test_samples, model)[1]


def get_input_sentences(start_idx):
  # Get whole sentences up to 1000 words; don't break sentences up
  end_idx = start_idx + 1000
  text = result[start_idx:end_idx]
  end_sentence_chars = [".", "?", "!"]
  next_start_idx = end_idx
  while text[-1] not in end_sentence_chars and end_idx < len(result):
      text = text[:-1]
      next_start_idx -= 1
  return text, next_start_idx

start = 0
input_text = []
summarized_text = []
while start < len(result):
  input, start = get_input_sentences(start)
  input_text.append(input)
  print("input text \n" + input)
  out = generate_inference_summary(input, model)[1]
  print("Summarized text\n", out)
  summarized_text.append(out)


print(summarized_text)

input text 
 Hi, this is Eric Johnson. It's February 18, 2021, and this is the engineering key review at GitLab. So I've got number four in the agenda, which is a proposal to break up this meeting into four department key reviews. So currently this is engineering. Development quality, security, and UX infrastructure and support to their own key reviews already. I have the reasons why increased visibility able to go deeper. Increased the objectivity with which my reports can manage their groups, allow me more time to focus on new markets and allow me to shift into more of a question asker mode, then generating content and answering questions in these meetings. And to avoid adding three net new meetings to stakeholders counters, I propose we do a sort of two month rotation. So month one development quality go month two security and UX would go. How do people feel about that proposal? I think in the group conversations it's working really well. So I'm supportive and this is the smallest t

In [39]:
summary = ""
for x in summarized_text:
    for y in x:
        summary += y

In [40]:
print(summary)

 The agenda for the first meeting of the engineering key review at GitLab has been agenda. I have to check the taxonomy for engineering. I believe the water MR rate just captures community contributions from the company. It's when you look at like the development and infrastructure will oftentimes contribute to developments work but it won't be counted as MRs. The chief executive of GitLab, Eric R&D, has told the company that the number of external contributors is number MRs per external contributor. The chief executive of F, the company that created a "pre-sophisticated taxonomy with prefixes and postfixes to talk about these things, but in reality, we've only two of them. It's what all our investors ask about the investor ask about what they ask about. I'm sorry, Max I was looking up to see if I had the percentage graph for the number of community MRs. I'm not quite sure whose responsibility is to focus on getting a handle on some of the constraints we have on replication. I did ask 